In [1]:
import os
os.environ["TF_GPU_ALLOCATOR"]="cuda_malloc_async"
import gc
import numpy as np

import tensorflow as tf

from merlin.schema.tags import Tags
from merlin.io.dataset import Dataset
import merlin.models.tf as mm

2023-03-13 20:26:22.114565: I tensorflow/core/platform/cpu_feature_guard.cc:194] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


/usr/local/lib/python3.8/dist-packages/merlin/dtypes/mappings/torch.py:43: UserWarning: PyTorch dtype mappings did not load successfully due to an error: No module named 'torch'
  warn(f"PyTorch dtype mappings did not load successfully due to an error: {exc.msg}")
2023-03-13 20:26:24.538242: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-13 20:26:24.538645: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-13 20:26:24.538803: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgr

In [2]:
train = Dataset("ecom_dataset/0001/train.parquet")
valid = Dataset("ecom_dataset/0002/test.parquet")

In [3]:
target = 'sess_pid_seq'

In [4]:
d_model = 192
n_layer = 3
n_head = 16
batch_size = 128
learning_rate = 0.0006667377132554976
n_epoch = 5
item_embedding_dim = 448 
item_id_embeddings_init_std = 3

In [5]:
def get_model():
    mlp_block = mm.MLPBlock(
                    [d_model],
                    activation='relu',
                    no_activation_last_layer=True,
                )

    from merlin.schema.io.tensorflow_metadata import TensorflowMetadata

    schema = TensorflowMetadata.from_proto_text_file(
        '../',
        file_name='rees46_schema_modified.pbtxt'
    ).to_merlin_schema()

    train.schema = schema

    schema_model = schema.select_by_tag(Tags.ITEM_ID)
    input_block = mm.InputBlockV2(
        schema_model,
        categorical=mm.Embeddings(
                schema_model.select_by_tag(Tags.CATEGORICAL),
                dim=item_embedding_dim,
                sequence_combiner=None,
            )
        )

    train.schema = train.schema.select_by_name('sess_pid_seq')

    xlnet_block = mm.XLNetBlock(d_model=d_model, n_head=n_head, n_layer=n_layer)

    dense_block = mm.SequentialBlock(
        input_block,
        mlp_block,
        xlnet_block
    )

    mlp_block2 = mm.MLPBlock(
                    [item_embedding_dim],
                    activation='relu',
                    no_activation_last_layer=True,
                )

    prediction_task = mm.CategoricalOutput(
        to_call=input_block["categorical"][target],
    )

    model_transformer = mm.Model(dense_block, mlp_block2, prediction_task)

    optimizer = tf.keras.optimizers.Adam(
        learning_rate=learning_rate,
    )

    model_transformer.compile(run_eagerly=False, optimizer=optimizer, loss="categorical_crossentropy",
                  metrics=mm.TopKMetricsAggregator.default_metrics(top_ks=[20])
                 )
    return model_transformer, xlnet_block

# Run 1

In [11]:
model_transformer, xlnet_block = get_model()
model_transformer.fit(
    train,
    batch_size=batch_size,
    epochs=n_epoch,
    pre=mm.SequenceMaskRandom(schema=train.schema, target=target, transformer=xlnet_block)
)

predict_last = mm.SequencePredictLast(schema=valid.schema, target=target, transformer=xlnet_block)
model_transformer.evaluate(
    valid,
    batch_size=batch_size,
    pre=predict_last,
    return_dict=True
)

Epoch 1/5


/usr/local/lib/python3.8/dist-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model_5/sequential_block_29/xl_net_block_5/prepare_transformer_inputs_4/RaggedToTensor_1/boolean_mask_1/GatherV2:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/model_5/sequential_block_29/xl_net_block_5/prepare_transformer_inputs_4/RaggedToTensor_1/boolean_mask/GatherV2:0", shape=(None, 192), dtype=float32), dense_shape=Tensor("gradient_tape/model_5/sequential_block_29/xl_net_block_5/prepare_transformer_inputs_4/RaggedToTensor_1/Shape:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model_5/sequential_block_29/xl_net_block_5/sequential_block_3

2023-03-14 06:41:41.374760: W tensorflow/core/grappler/optimizers/loop_optimizer.cc:907] Skipping loop optimization for Merge node with control input: model_5/sequential_block_29/xl_net_block_5/sequential_block_32/replace_masked_embeddings_5/RaggedWhere/Assert/AssertGuard/branch_executed/_31


677/677 [==============================] - 164s 230ms/step - loss: 9.4924 - recall_at_20: 0.0704 - mrr_at_20: 0.0174 - ndcg_at_20: 0.0288 - map_at_20: 0.0174 - precision_at_20: 0.0035 - regularization_loss: 0.0000e+00 - loss_batch: 9.4899
Epoch 2/5
677/677 [==============================] - 159s 234ms/step - loss: 8.0755 - recall_at_20: 0.1759 - mrr_at_20: 0.0480 - ndcg_at_20: 0.0758 - map_at_20: 0.0480 - precision_at_20: 0.0088 - regularization_loss: 0.0000e+00 - loss_batch: 8.0726
Epoch 3/5
677/677 [==============================] - 159s 235ms/step - loss: 7.3926 - recall_at_20: 0.2427 - mrr_at_20: 0.0671 - ndcg_at_20: 0.1053 - map_at_20: 0.0671 - precision_at_20: 0.0121 - regularization_loss: 0.0000e+00 - loss_batch: 7.3887
Epoch 4/5
677/677 [==============================] - 159s 235ms/step - loss: 6.9299 - recall_at_20: 0.2932 - mrr_at_20: 0.0821 - ndcg_at_20: 0.1281 - map_at_20: 0.0821 - precision_at_20: 0.0147 - regularization_loss: 0.0000e+00 - loss_batch: 6.9255
Epoch 5/5
677/

2023-03-14 06:54:43.265476: W tensorflow/core/grappler/optimizers/loop_optimizer.cc:907] Skipping loop optimization for Merge node with control input: model_5/sequential_block_29/xl_net_block_5/sequential_block_32/replace_masked_embeddings_5/RaggedWhere/Assert/AssertGuard/branch_executed/_529


84/84 [==============================] - 9s 49ms/step - loss: 8.3507 - recall_at_20: 0.2332 - mrr_at_20: 0.0720 - ndcg_at_20: 0.1070 - map_at_20: 0.0720 - precision_at_20: 0.0117 - regularization_loss: 0.0000e+00 - loss_batch: 8.3848


{'loss': 8.350717544555664,
 'recall_at_20': 0.23180365562438965,
 'mrr_at_20': 0.06943727284669876,
 'ndcg_at_20': 0.10483581572771072,
 'map_at_20': 0.06943727284669876,
 'precision_at_20': 0.011590182781219482,
 'regularization_loss': 0.0,
 'loss_batch': 9.85844612121582}

In [12]:
model_transformer.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_block_29 (Sequen  multiple                 176252800 
 tialBlock)                                                      
                                                                 
 sequential_block_30 (Sequen  multiple                 86464     
 tialBlock)                                                      
                                                                 
 sess_pid_seq/categorical_ou  multiple                 175110449 
 tput (CategoricalOutput)                                        
                                                                 
 model_context_5 (ModelConte  multiple                 0         
 xt)                                                             
                                                                 
 prepare_features_11 (Prepar  multiple                 0   

# Run 2

In [7]:
model_transformer, xlnet_block = get_model()
model_transformer.fit(
    train,
    batch_size=batch_size,
    epochs=n_epoch,
    pre=mm.SequenceMaskRandom(schema=train.schema, target=target, transformer=xlnet_block)
)

predict_last = mm.SequencePredictLast(schema=valid.schema, target=target, transformer=xlnet_block)
model_transformer.evaluate(
    valid,
    batch_size=batch_size,
    pre=predict_last,
    return_dict=True
)

/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:149: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(


Epoch 1/5


/usr/local/lib/python3.8/dist-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model_1/sequential_block_9/xl_net_block_1/prepare_transformer_inputs_4/RaggedToTensor_1/boolean_mask_1/GatherV2:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/model_1/sequential_block_9/xl_net_block_1/prepare_transformer_inputs_4/RaggedToTensor_1/boolean_mask/GatherV2:0", shape=(None, 192), dtype=float32), dense_shape=Tensor("gradient_tape/model_1/sequential_block_9/xl_net_block_1/prepare_transformer_inputs_4/RaggedToTensor_1/Shape:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model_1/sequential_block_9/xl_net_block_1/sequential_block_12/re

2023-03-13 20:31:46.363004: W tensorflow/core/grappler/optimizers/loop_optimizer.cc:907] Skipping loop optimization for Merge node with control input: model_1/sequential_block_9/xl_net_block_1/sequential_block_12/replace_masked_embeddings_1/RaggedWhere/Assert/AssertGuard/branch_executed/_31


677/677 [==============================] - 65s 84ms/step - loss: 9.4953 - recall_at_20: 0.0663 - mrr_at_20: 0.0167 - ndcg_at_20: 0.0274 - map_at_20: 0.0167 - precision_at_20: 0.0033 - regularization_loss: 0.0000e+00 - loss_batch: 9.4908
Epoch 2/5
677/677 [==============================] - 57s 84ms/step - loss: 8.1077 - recall_at_20: 0.1712 - mrr_at_20: 0.0474 - ndcg_at_20: 0.0744 - map_at_20: 0.0474 - precision_at_20: 0.0086 - regularization_loss: 0.0000e+00 - loss_batch: 8.1021
Epoch 3/5
677/677 [==============================] - 57s 84ms/step - loss: 7.3969 - recall_at_20: 0.2444 - mrr_at_20: 0.0671 - ndcg_at_20: 0.1057 - map_at_20: 0.0671 - precision_at_20: 0.0122 - regularization_loss: 0.0000e+00 - loss_batch: 7.3975
Epoch 4/5
677/677 [==============================] - 57s 84ms/step - loss: 6.9683 - recall_at_20: 0.2853 - mrr_at_20: 0.0794 - ndcg_at_20: 0.1243 - map_at_20: 0.0794 - precision_at_20: 0.0143 - regularization_loss: 0.0000e+00 - loss_batch: 6.9657
Epoch 5/5
677/677 [===

2023-03-13 20:36:37.576034: W tensorflow/core/grappler/optimizers/loop_optimizer.cc:907] Skipping loop optimization for Merge node with control input: model_1/sequential_block_9/xl_net_block_1/sequential_block_12/replace_masked_embeddings_1/RaggedWhere/Assert/AssertGuard/branch_executed/_529


84/84 [==============================] - 9s 48ms/step - loss: 8.3509 - recall_at_20: 0.2300 - mrr_at_20: 0.0691 - ndcg_at_20: 0.1041 - map_at_20: 0.0691 - precision_at_20: 0.0115 - regularization_loss: 0.0000e+00 - loss_batch: 8.3545


{'loss': 8.350946426391602,
 'recall_at_20': 0.22926461696624756,
 'mrr_at_20': 0.06758848577737808,
 'ndcg_at_20': 0.10286629945039749,
 'map_at_20': 0.06758848577737808,
 'precision_at_20': 0.011463231407105923,
 'regularization_loss': 0.0,
 'loss_batch': 8.509391784667969}

# Run 3

In [8]:
model_transformer, xlnet_block = get_model()
model_transformer.fit(
    train,
    batch_size=batch_size,
    epochs=n_epoch,
    pre=mm.SequenceMaskRandom(schema=train.schema, target=target, transformer=xlnet_block)
)

predict_last = mm.SequencePredictLast(schema=valid.schema, target=target, transformer=xlnet_block)
model_transformer.evaluate(
    valid,
    batch_size=batch_size,
    pre=predict_last,
    return_dict=True
)

Epoch 1/5


/usr/local/lib/python3.8/dist-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model_2/sequential_block_14/xl_net_block_2/prepare_transformer_inputs_4/RaggedToTensor_1/boolean_mask_1/GatherV2:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/model_2/sequential_block_14/xl_net_block_2/prepare_transformer_inputs_4/RaggedToTensor_1/boolean_mask/GatherV2:0", shape=(None, 192), dtype=float32), dense_shape=Tensor("gradient_tape/model_2/sequential_block_14/xl_net_block_2/prepare_transformer_inputs_4/RaggedToTensor_1/Shape:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model_2/sequential_block_14/xl_net_block_2/sequential_block_1

2023-03-13 20:36:51.268625: W tensorflow/core/grappler/optimizers/loop_optimizer.cc:907] Skipping loop optimization for Merge node with control input: model_2/sequential_block_14/xl_net_block_2/sequential_block_17/replace_masked_embeddings_2/RaggedWhere/Assert/AssertGuard/branch_executed/_31


677/677 [==============================] - 65s 84ms/step - loss: 9.4386 - recall_at_20: 0.0722 - mrr_at_20: 0.0190 - ndcg_at_20: 0.0305 - map_at_20: 0.0190 - precision_at_20: 0.0036 - regularization_loss: 0.0000e+00 - loss_batch: 9.4342
Epoch 2/5
677/677 [==============================] - 57s 84ms/step - loss: 8.0171 - recall_at_20: 0.1837 - mrr_at_20: 0.0502 - ndcg_at_20: 0.0792 - map_at_20: 0.0502 - precision_at_20: 0.0092 - regularization_loss: 0.0000e+00 - loss_batch: 8.0103
Epoch 3/5
677/677 [==============================] - 58s 85ms/step - loss: 7.3722 - recall_at_20: 0.2467 - mrr_at_20: 0.0691 - ndcg_at_20: 0.1078 - map_at_20: 0.0691 - precision_at_20: 0.0123 - regularization_loss: 0.0000e+00 - loss_batch: 7.3658
Epoch 4/5
677/677 [==============================] - 57s 85ms/step - loss: 6.9592 - recall_at_20: 0.2892 - mrr_at_20: 0.0807 - ndcg_at_20: 0.1262 - map_at_20: 0.0807 - precision_at_20: 0.0145 - regularization_loss: 0.0000e+00 - loss_batch: 6.9549
Epoch 5/5
677/677 [===

2023-03-13 20:41:42.865959: W tensorflow/core/grappler/optimizers/loop_optimizer.cc:907] Skipping loop optimization for Merge node with control input: model_2/sequential_block_14/xl_net_block_2/sequential_block_17/replace_masked_embeddings_2/RaggedWhere/Assert/AssertGuard/branch_executed/_529


84/84 [==============================] - 9s 49ms/step - loss: 8.3114 - recall_at_20: 0.2264 - mrr_at_20: 0.0687 - ndcg_at_20: 0.1030 - map_at_20: 0.0687 - precision_at_20: 0.0113 - regularization_loss: 0.0000e+00 - loss_batch: 8.3190


{'loss': 8.311356544494629,
 'recall_at_20': 0.22738386690616608,
 'mrr_at_20': 0.0663006603717804,
 'ndcg_at_20': 0.10139463096857071,
 'map_at_20': 0.0663006603717804,
 'precision_at_20': 0.011369192972779274,
 'regularization_loss': 0.0,
 'loss_batch': 8.649133682250977}

# Run 4

In [9]:
model_transformer, xlnet_block = get_model()
model_transformer.fit(
    train,
    batch_size=batch_size,
    epochs=n_epoch,
    pre=mm.SequenceMaskRandom(schema=train.schema, target=target, transformer=xlnet_block)
)

predict_last = mm.SequencePredictLast(schema=valid.schema, target=target, transformer=xlnet_block)
model_transformer.evaluate(
    valid,
    batch_size=batch_size,
    pre=predict_last,
    return_dict=True
)

Epoch 1/5


/usr/local/lib/python3.8/dist-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model_3/sequential_block_19/xl_net_block_3/prepare_transformer_inputs_4/RaggedToTensor_1/boolean_mask_1/GatherV2:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/model_3/sequential_block_19/xl_net_block_3/prepare_transformer_inputs_4/RaggedToTensor_1/boolean_mask/GatherV2:0", shape=(None, 192), dtype=float32), dense_shape=Tensor("gradient_tape/model_3/sequential_block_19/xl_net_block_3/prepare_transformer_inputs_4/RaggedToTensor_1/Shape:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model_3/sequential_block_19/xl_net_block_3/sequential_block_2

2023-03-13 20:41:56.776497: W tensorflow/core/grappler/optimizers/loop_optimizer.cc:907] Skipping loop optimization for Merge node with control input: model_3/sequential_block_19/xl_net_block_3/sequential_block_22/replace_masked_embeddings_3/RaggedWhere/Assert/AssertGuard/branch_executed/_31


677/677 [==============================] - 65s 84ms/step - loss: 9.4786 - recall_at_20: 0.0673 - mrr_at_20: 0.0176 - ndcg_at_20: 0.0283 - map_at_20: 0.0176 - precision_at_20: 0.0034 - regularization_loss: 0.0000e+00 - loss_batch: 9.4794
Epoch 2/5
677/677 [==============================] - 57s 84ms/step - loss: 8.1173 - recall_at_20: 0.1692 - mrr_at_20: 0.0454 - ndcg_at_20: 0.0723 - map_at_20: 0.0454 - precision_at_20: 0.0085 - regularization_loss: 0.0000e+00 - loss_batch: 8.1128
Epoch 3/5
677/677 [==============================] - 57s 85ms/step - loss: 7.4296 - recall_at_20: 0.2409 - mrr_at_20: 0.0664 - ndcg_at_20: 0.1044 - map_at_20: 0.0664 - precision_at_20: 0.0120 - regularization_loss: 0.0000e+00 - loss_batch: 7.4268
Epoch 4/5
677/677 [==============================] - 58s 85ms/step - loss: 6.9533 - recall_at_20: 0.2861 - mrr_at_20: 0.0778 - ndcg_at_20: 0.1232 - map_at_20: 0.0778 - precision_at_20: 0.0143 - regularization_loss: 0.0000e+00 - loss_batch: 6.9502
Epoch 5/5
677/677 [===

2023-03-13 20:46:48.752036: W tensorflow/core/grappler/optimizers/loop_optimizer.cc:907] Skipping loop optimization for Merge node with control input: model_3/sequential_block_19/xl_net_block_3/sequential_block_22/replace_masked_embeddings_3/RaggedWhere/Assert/AssertGuard/branch_executed/_529


84/84 [==============================] - 9s 49ms/step - loss: 8.3535 - recall_at_20: 0.2280 - mrr_at_20: 0.0700 - ndcg_at_20: 0.1046 - map_at_20: 0.0700 - precision_at_20: 0.0114 - regularization_loss: 0.0000e+00 - loss_batch: 8.3763


{'loss': 8.353541374206543,
 'recall_at_20': 0.23067519068717957,
 'mrr_at_20': 0.06726308912038803,
 'ndcg_at_20': 0.10282379388809204,
 'map_at_20': 0.06726308912038803,
 'precision_at_20': 0.011533760465681553,
 'regularization_loss': 0.0,
 'loss_batch': 9.360955238342285}

# Run 5

In [10]:
model_transformer, xlnet_block = get_model()
model_transformer.fit(
    train,
    batch_size=batch_size,
    epochs=n_epoch,
    pre=mm.SequenceMaskRandom(schema=train.schema, target=target, transformer=xlnet_block)
)

predict_last = mm.SequencePredictLast(schema=valid.schema, target=target, transformer=xlnet_block)
model_transformer.evaluate(
    valid,
    batch_size=batch_size,
    pre=predict_last,
    return_dict=True
)

Epoch 1/5


/usr/local/lib/python3.8/dist-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model_4/sequential_block_24/xl_net_block_4/prepare_transformer_inputs_4/RaggedToTensor_1/boolean_mask_1/GatherV2:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/model_4/sequential_block_24/xl_net_block_4/prepare_transformer_inputs_4/RaggedToTensor_1/boolean_mask/GatherV2:0", shape=(None, 192), dtype=float32), dense_shape=Tensor("gradient_tape/model_4/sequential_block_24/xl_net_block_4/prepare_transformer_inputs_4/RaggedToTensor_1/Shape:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model_4/sequential_block_24/xl_net_block_4/sequential_block_2

2023-03-13 20:47:02.588234: W tensorflow/core/grappler/optimizers/loop_optimizer.cc:907] Skipping loop optimization for Merge node with control input: model_4/sequential_block_24/xl_net_block_4/sequential_block_27/replace_masked_embeddings_4/RaggedWhere/Assert/AssertGuard/branch_executed/_31


677/677 [==============================] - 65s 84ms/step - loss: 9.4909 - recall_at_20: 0.0707 - mrr_at_20: 0.0184 - ndcg_at_20: 0.0297 - map_at_20: 0.0184 - precision_at_20: 0.0035 - regularization_loss: 0.0000e+00 - loss_batch: 9.4882
Epoch 2/5
677/677 [==============================] - 57s 84ms/step - loss: 8.1387 - recall_at_20: 0.1653 - mrr_at_20: 0.0453 - ndcg_at_20: 0.0713 - map_at_20: 0.0453 - precision_at_20: 0.0083 - regularization_loss: 0.0000e+00 - loss_batch: 8.1347
Epoch 3/5
677/677 [==============================] - 57s 84ms/step - loss: 7.4398 - recall_at_20: 0.2387 - mrr_at_20: 0.0662 - ndcg_at_20: 0.1038 - map_at_20: 0.0662 - precision_at_20: 0.0119 - regularization_loss: 0.0000e+00 - loss_batch: 7.4371
Epoch 4/5
677/677 [==============================] - 57s 85ms/step - loss: 6.9831 - recall_at_20: 0.2878 - mrr_at_20: 0.0810 - ndcg_at_20: 0.1261 - map_at_20: 0.0810 - precision_at_20: 0.0144 - regularization_loss: 0.0000e+00 - loss_batch: 6.9787
Epoch 5/5
677/677 [===

2023-03-13 20:51:54.265885: W tensorflow/core/grappler/optimizers/loop_optimizer.cc:907] Skipping loop optimization for Merge node with control input: model_4/sequential_block_24/xl_net_block_4/sequential_block_27/replace_masked_embeddings_4/RaggedWhere/Assert/AssertGuard/branch_executed/_529


84/84 [==============================] - 9s 49ms/step - loss: 8.3766 - recall_at_20: 0.2347 - mrr_at_20: 0.0690 - ndcg_at_20: 0.1050 - map_at_20: 0.0690 - precision_at_20: 0.0117 - regularization_loss: 0.0000e+00 - loss_batch: 8.3785


{'loss': 8.376553535461426,
 'recall_at_20': 0.23227383196353912,
 'mrr_at_20': 0.0675581842660904,
 'ndcg_at_20': 0.10343420505523682,
 'map_at_20': 0.0675581842660904,
 'precision_at_20': 0.011613693088293076,
 'regularization_loss': 0.0,
 'loss_batch': 8.46284294128418}